<img src="../img/htw-logo.png" width=150>


**I758 Wissens- und KI-basierte Systeme**

# Churn Prediction mit Neuronalen Netzwerken
 

<font color="green"><b>KLAUSURTAUGLICH.</b></font>
Dieses Notebook gehört zu den fünf Notebooks, die Sie für die Klausur einreichen können. Bei vollständiger und korrekter Bearbeitung **erhalten Sie Punkte für die Abgabe, die zu Ihrer Klausur addiert werden.**




Wir arbeiten heute mit einem weiteren "Klassiker" als Datensatz: dem IBM Customer Churn data set. Eine Dokumentation dazu können Sie [hier](https://www.ibm.com/docs/en/cognos-analytics/11.1.0?topic=samples-telco-customer-churn) finden. Es geht darum, basierend auf Kundendaten von Kunden einer (fiktiven) Telekom-Firma vorherzusagen, welcher Kunde im nächsten Monat seinen Vertrag beendet. Wir arbeiten also mit einem Klassikations-Problem: welche Kunden "churnen" (und benötigen deshalb besondere Aufmerksamkeit, z.B. einen freundlichen Anruf) und welche nicht?

Diesmal wollen wir das Verhalten mit Hilfe eines neuronalen Netzwerkes vorhersagen. Dazu nutzen wir das extrem mächtige und umfangreiche Tensorflow-Framework mit der Keras-Bibliothek.

In [1]:
# das hier könnte länger dauern (bis zu 10 Minuten?)
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model

Die Daten sind online frei verfügbar, liegen der Einfachheit halber aber im Verzeichnis vor. Wir sehen Sie uns etwas im Detail an.

In [3]:
data = pd.read_csv('data/Churn_Customers.csv')

In [4]:
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
data.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

Neuronale Netze sind von Natur aus rein numerisch arbeitende Modelle. Kategorische Variablen oder ähnliches sind nicht möglich. Versuchen wir, die Daten für unser numerisches Modell vorzubereiten. Gleich am Anfang erleben wir den ersten kleinen Stolperstein. Senior Citizen ist eine kategorische Variable - wird aber numerisch codiert. Das lässt sich (eigentlich) besser formulieren. Wir machen das Feature deshalb wieder kategorisch.

In [6]:
data.SeniorCitizen.replace([0, 1], ["No", "Yes"], inplace= True)

Die folgende Zeile könnte Probleme machen:

In [7]:
data.TotalCharges = data.TotalCharges.astype(float)

ValueError: could not convert string to float: ' '

<div class="alert alert-block alert-success">
<b>Arbeitsauftrag:</b> 
Nanu? Was ist hier los? Finden Sie es heraus:
</div>

In [17]:
import pandas as pd
import numpy as np

data = pd.read_csv('data/Churn_Customers.csv')

df = pd.DataFrame(data)
df['TotalCharges'] = df['TotalCharges'].replace('', np.nan)
print(df)

      customerID  gender  SeniorCitizen Partner Dependents  tenure  \
0     7590-VHVEG  Female              0     Yes         No       1   
1     5575-GNVDE    Male              0      No         No      34   
2     3668-QPYBK    Male              0      No         No       2   
3     7795-CFOCW    Male              0      No         No      45   
4     9237-HQITU  Female              0      No         No       2   
...          ...     ...            ...     ...        ...     ...   
7038  6840-RESVB    Male              0     Yes        Yes      24   
7039  2234-XADUH  Female              0     Yes        Yes      72   
7040  4801-JZAZL  Female              0     Yes        Yes      11   
7041  8361-LTMKD    Male              1     Yes         No       4   
7042  3186-AJIEK    Male              0      No         No      66   

     PhoneService     MultipleLines InternetService OnlineSecurity  ...  \
0              No  No phone service             DSL             No  ...   
1        

In [ ]:
for i in range(len(data)):
  if data.TotalCharges[i] == " ":
      print("Tenure is %s and Monthly charges are %s" % (data.tenure[i], data.MonthlyCharges[i]))

Beschreiben Sie hier das Problem... die leeren Zeichenketten bestehen aus Leerzeichen und nicht aus leeren Zeichenketten.

<div class="alert alert-block alert-success">
<b>Arbeitsauftrag:</b> 
Was sind das für Kunden? Lassen Sie uns die Daten reparieren und nach float konvertieren.
</div>

In [16]:
import pandas as pd
import numpy as np

data = pd.read_csv('data/Churn_Customers.csv')

df = pd.DataFrame(data)
df['TotalCharges'] = df['TotalCharges'].replace(['', ' '], np.nan)
df['TotalCharges'] = df['TotalCharges'].astype(float)

print(df)

      customerID  gender  SeniorCitizen Partner Dependents  tenure  \
0     7590-VHVEG  Female              0     Yes         No       1   
1     5575-GNVDE    Male              0      No         No      34   
2     3668-QPYBK    Male              0      No         No       2   
3     7795-CFOCW    Male              0      No         No      45   
4     9237-HQITU  Female              0      No         No       2   
...          ...     ...            ...     ...        ...     ...   
7038  6840-RESVB    Male              0     Yes        Yes      24   
7039  2234-XADUH  Female              0     Yes        Yes      72   
7040  4801-JZAZL  Female              0     Yes        Yes      11   
7041  8361-LTMKD    Male              1     Yes         No       4   
7042  3186-AJIEK    Male              0      No         No      66   

     PhoneService     MultipleLines InternetService OnlineSecurity  ...  \
0              No  No phone service             DSL             No  ...   
1        

<div class="alert alert-block alert-success">
<b>Arbeitsauftrag:</b> 
Customer ID sollte eigentlich keinen Informationsgehalt haben - dieses Feature kann weg. Schmeißen Sie es aus dem Datensatz.
</div>

In [19]:
import pandas as pd

data = pd.read_csv('data/Churn_Customers.csv')
data.drop('customerID', axis=1, inplace=True)
print(data)

      gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0     Female              0     Yes         No       1           No   
1       Male              0      No         No      34          Yes   
2       Male              0      No         No       2          Yes   
3       Male              0      No         No      45           No   
4     Female              0      No         No       2          Yes   
...      ...            ...     ...        ...     ...          ...   
7038    Male              0     Yes        Yes      24          Yes   
7039  Female              0     Yes        Yes      72          Yes   
7040  Female              0     Yes        Yes      11           No   
7041    Male              1     Yes         No       4          Yes   
7042    Male              0      No         No      66          Yes   

         MultipleLines InternetService OnlineSecurity OnlineBackup  \
0     No phone service             DSL             No          Yes   
1      

Das sollte für den Anfang reichen. Welche Werte bleiben übrig?

In [20]:
for col in data.dtypes[data.dtypes == object].index:
    print(col, data[col].unique())

gender ['Female' 'Male']
Partner ['Yes' 'No']
Dependents ['No' 'Yes']
PhoneService ['No' 'Yes']
MultipleLines ['No phone service' 'No' 'Yes']
InternetService ['DSL' 'Fiber optic' 'No']
OnlineSecurity ['No' 'Yes' 'No internet service']
OnlineBackup ['Yes' 'No' 'No internet service']
DeviceProtection ['No' 'Yes' 'No internet service']
TechSupport ['No' 'Yes' 'No internet service']
StreamingTV ['No' 'Yes' 'No internet service']
StreamingMovies ['No' 'Yes' 'No internet service']
Contract ['Month-to-month' 'One year' 'Two year']
PaperlessBilling ['Yes' 'No']
PaymentMethod ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
TotalCharges ['29.85' '1889.5' '108.15' ... '346.45' '306.6' '6844.5']
Churn ['No' 'Yes']


Auch unsere Zielfunktion kann leider nicht kategorisch sein, deshalb ersetzen wir die Werte durch Integer:

In [21]:
data.Churn.replace(["Yes", "No"], [1, 0], inplace= True)

Jetzt nutzen wir noch das bekannte One Hot Encoding, um die restlichen kategorischen Werte zu transformieren.

In [22]:
data = pd.get_dummies(data)

Dieser Teil sollte Ihnen sehr bekannt vorkommen. 

In [23]:
X = data.drop("Churn", axis= 1)
y = data.Churn

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 1234)

Hier wird es spannend: so bauen wir das neuronale Netz aus mehreren Layers. Studieren Sie den Code und folgen Sie dem Ablauf bis zum Berechnen der Accuracy.

In [24]:
model = Sequential()
model.add(Dense(16, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [25]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Das hier könnte etwas dauern (ein paar Minuten)

In [ ]:
model.fit(X_train, y_train, epochs=150, batch_size=10)

In [ ]:
_, accuracy = model.evaluate(X_test, y_test)
accuracy

<div class="alert alert-block alert-success">
<b>Arbeitsauftrag:</b> 
Versuchen Sie nun, dem Modell eine ganz andere Struktur (mehr/weniger Layer, größere/kleinere/andere Layer) zu geben oder das Training zu variieren. Vielleicht wollen Sie sich auch zu <a href="https://keras.io/guides/sequential_model/">Sequential Models</a> belesen. Versuchen Sie, die ursprüngliche Modell-Accuracy des Originals von 78,64 Prozent (Achtung: auf den Testdaten) zu übertreffen!
</div>

Zum Schluss können Sie Ihr Modell speichern:

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

data = pd.read_csv('data/Churn_Customers.csv')

numerical_columns = ['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges']
X = data[numerical_columns]
X.loc[:, 'TotalCharges'] = pd.to_numeric(X['TotalCharges'], errors='coerce')
X = X.dropna()
y = data['Churn'].loc[X.index]
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=1000, random_state=42)
model.fit(X_train, y_train)

accuracy = model.score(X_test, y_test)
print(f'Model Accuracy: {accuracy}')


Model Accuracy: 0.7775408670931059
